### Code Hist.

- CODE  
  &ensp; : KIER Data_단순 분리 (전처리 X)  

- DESC  
  &emsp; [수행 작업]  
  &emsp; 1) Date 형식 검사 및 이상치 처리  
  &emsp;&emsp; 1-1) Datetime 유효성 확인  
  &emsp;&emsp; 1-2) Datetime 이상치 제거  
  &emsp; 2) 세대별 사용량 분리  
  &emsp;&emsp; 2-1) HOUSE_ID를 기반으로 적산 사용량 분리  
  &emsp;&emsp; 2-2) 세대별 적산 사용량을 .csv 파일로 저장  
  &emsp;&emsp; 2-3) 세대별 분리된 적산 사용량을 Combine  

- DATE  
  &ensp; 2023-10-12 Created  
  &ensp; 2023-11-21 Code 최신화  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 기존 모듈화된 함수 사용  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 기존 KIER 변환 관련 코드를 모두 통합  
  &ensp; 2024-01-19 Code 개선  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) HOT (온수 사용량) 전용 Code 작성  
  &ensp; 2024-03-13 Code 개선  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
  &ensp; 2024-06-27 Code 개선  
  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) GAS Domain 추가    

## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

#### Module import

In [1]:
#region Basic_Import
## Basic
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series

import math, random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# CLustering 알고리즘의 성능 평가 측도
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm
import tqdm

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential, load_model
    from keras_flops import get_flops
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    print('Using GPU')
    tf.config.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)
else : 
  print('Using CPU')

Using GPU


In [4]:
## Import_Local
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
# from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
# from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
# from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
# from Src_Dev_Common import DEV_Common_Model as com_Model
# from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
# from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
# from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC
# from Src_Dev_Common import DEV_KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [5]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [6]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-07-02 19:19:02.349221
2024 / 7 / 2
19 : 19


In [7]:
## Dict_Domain
dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 5:"GAS"} ## GAS는 사용하지 않음.
int_domain = 3
str_domain = str(dict_domain[int_domain])

dict_col_accu = {0 : "ACTUAL_ACCU_EFF" ## ELEC
                 , 1 : "ACCU_HEAT" ## HEAT
                 , 2 : "ACCU_FLOW" ## WATER
                 , 3 : "ACCU" ## HOT 열량
                 , 4 : "ACCU" ## HOT 유량
                 , 5 : "ACCU_FLOW"} ## GAS
str_col_accu = str(str_domain + "_" + str(dict_col_accu[int_domain]))

dict_col_inst = {0 : "INST_EFF" ## ELEC_INST_EFF
                , 1 : "INST_HEAT" ## HEAT_INST_HEAT
                , 2 : "INST_FLOW" ## WATER_INST_FLOW
                , 3 : "INST" ## HOT_HEAT_INST
                , 4 : "INST" ## HOT_FLOW_INST
                , 5 : "INST_FLOW"} ## GAS_INST_FLOW
str_col_inst = str(str_domain + "_" + str(dict_col_inst[int_domain]))

## Directory Root
str_dirData = "../data/data_Energy_KIER/"
str_dir_raw = '../data/data_Energy_KIER/KIER_0_Raw/'
str_dirName_bld = '../data/data_Energy_KIER/KIER_1_BLD/'
str_dirName_h = '../data/data_Energy_KIER/KIER_2_H_' + str_domain + '/'

## File
str_fileRaw = str('KIER_RAW_' + str_domain + '_2024-06-07.csv')
str_fileRaw_hList = str('KIER_hList_Common.csv') 

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_BLD', 'KIER_2_H_ELEC', 'KIER_2_H_GAS', 'KIER_2_H_HEAT', 'KIER_2_H_HOT_FLOW', 'KIER_2_H_HOT_HEAT', 'KIER_2_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_HOT_HEAT_561-1-1_ACCU_01_Raw.csv', 'KIER_HOT_HEAT_561-1-1_INST_01_10min.csv', 'KIER_HOT_HEAT_561-1-2_ACCU_01_Raw.csv', 'KIER_HOT_HEAT_561-1-2_INST_01_10min.csv', 'KIER_HOT_HEAT_561-1-3_ACCU_01_Raw.csv', 'KIER_HOT_HEAT_561-1-3_INST_01_10min.csv', 'KIER_HOT_HEAT_561-1-4_ACCU_01_Raw.csv', 'KIER_HOT_HEAT_561-1-4_INST_01_10min.csv', 'KIER_HOT_HEAT_561-10-1_ACCU_01_Raw.csv', 'KIER_HOT_HEAT_561-10-1_INST_01_10min.csv', 'KIER_HOT_HEAT_561

In [8]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
list_Bld = df_kier_hList['HOUSE_ID_DONG'].drop_duplicates()
print(df_kier_hList.columns)
df_kier_hList

Index(['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID',
       'HOUSE_ID_HO'],
      dtype='object')


,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID,HOUSE_ID_HO
1,561,1,1,561-1-1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=
2,561,1,2,561-1-2,ODBEOTE0QzZCMjg5RUMzNTRBQjM1NjE0RDNBNDE0NzU=
3,561,1,3,561-1-3,QTUwRUJDQTlBMkZENUQyNjg0RDRGREVFMTNGNDI3NUY=
4,561,1,4,561-1-4,RkRCQzY4OTlCMTRDQjgzNUZBREE3Njc2MkM2MUM2QjI=
5,561,2,1,561-2-1,MTQxNkQ1RDAzOTE5NTM5RUM3MzlDQkY3RTI2NERCQTM=
...,...,...,...,...,...
344,563,23,2,563-23-2,QkJCQjQwOThFNjgwNjUwQzJGRUJFNDlCNTREOEUxNDU=
345,563,23,3,563-23-3,QUJENzFFMEFDRjM4MjY3NTRDQzJCRjdDNUQ3ODA0N0Y=
346,563,23,4,563-23-4,RUVENzZDNTc4MUY4NDg5MUREQjZEOUM1M0Q2RjYzMDE=
347,563,24,1,563-24-1,NEVENDc1OTFFNDBBRjBDQzkzMTlEMDgzMkMzMkZFQTg=


In [9]:
str_file = 'KIER_' + str_domain + '_ACCU_10MIN.csv'       
df_kier_extract = pd.read_csv(str_dirName_h + str_file)
df_kier_extract.isna().sum()

Unnamed: 0                   0
YEAR                         0
MONTH                        0
DAY                          0
HOUR                         0
                          ... 
HOT_HEAT_ACCU_563-23-4    2603
HOT_HEAT_ACCU_563-24-1    2628
HOT_HEAT_ACCU_563-24-2    2579
MEAN_OF_ACCU              1609
SUM_OF_ACCU                  0
Length: 356, dtype: int64

## 01-02. Data Load (df_raw)

### 01-02-01. KIER (Energy Usage)

In [10]:
## "METER_DATE" Cleansing이 완료되지 않은 경우
## Gas의 경우는 ParserError: Error tokenizing data 발생하므로 아래와 같이 Load
df_raw = pd.read_csv(str_dir_raw + str_fileRaw, index_col= 0, error_bad_lines = False).reset_index()
## 단, warn_bad_lines는 True로 그대로 두어 어느 행이 사라졌는지 확인해야만 함

print(df_raw.shape, " /// ", df_raw.columns)
df_raw

(33807415, 6)  ///  Index(['index', 'METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE',
       'HOUSE_ID_HO', 'HOT_HEAT_ACCU'],
      dtype='object')


,index,METER_DATE,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO,HOT_HEAT_ACCU
0,0,2022-07-17 23:20:06,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
1,1,2022-07-17 23:30:06,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
2,2,2022-07-17 23:40:06,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
3,3,2022-07-17 23:50:06,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
4,4,2022-07-18 00:00:06,561,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
...,...,...,...,...,...,...
33807410,33807410,2024-06-05 14:20:00,563,24,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807411,33807411,2024-06-05 14:30:00,563,24,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807412,33807412,2024-06-05 14:40:00,563,24,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807413,33807413,2024-06-05 14:50:00,563,24,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000


In [11]:
## 1) Date Column에 대한 유효성 검사 및 이상 Data에 대한 소거
## 1-1) Datetime 유효성 확인
list_errValues = com_date.list_invalidDate(df_raw, 'METER_DATE')
print(len(list_errValues))
## 1-2) Datetime 이상치 제거
for i in range(len(list_errValues) - 1, -1, -1):
    df_raw = df_raw.drop(index = list_errValues[i], axis = 0)

print(df_raw.shape)

(33807415, 6)
50
[13796577, 13817649, 13818113, 13818114, 13818115, 13818116, 13818117, 13818118, 13818119, 13818120, 13818121, 13818122, 13818123, 13818124, 13818125, 13818126, 13818127, 13818128, 13818129, 13821437, 13865856, 13866954, 13866955, 13866956, 13866957, 13866958, 13866959, 13866960, 13866961, 13866962, 13866963, 13866964, 13866965, 13866966, 13866967, 13866968, 13866969, 13866970, 13866971, 13866972, 13866973, 13866974, 13866975, 13866976, 13870340, 13870366, 13886556, 13886557, 13886558, 13886559]
50
(33807365, 6)


In [12]:
## List의 'HOUSE_ID_HO_INT' Column을 df_raw에 Merge
df_raw = df_raw[['METER_DATE', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO'
                , str_col_accu]]

df_raw = pd.merge(df_raw, df_kier_hList
                  , how = 'left', on = ['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO'])
df_raw = df_raw[['METER_DATE', 'HOUSE_ID', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID_HO'
                , str_col_accu]]

str_fileCleansed = str('KIER_RAW_' + str_domain + '_Cleansed.csv')
df_raw.to_csv(str_dir_raw + str_fileCleansed)
df_raw

,METER_DATE,HOUSE_ID,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID_HO,HOT_HEAT_ACCU
0,2022-07-17 23:20:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
1,2022-07-17 23:30:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
2,2022-07-17 23:40:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
3,2022-07-17 23:50:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
4,2022-07-18 00:00:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
...,...,...,...,...,...,...,...
33807360,2024-06-05 14:20:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807361,2024-06-05 14:30:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807362,2024-06-05 14:40:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807363,2024-06-05 14:50:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000


In [13]:
## "METER_DATE" Cleansing이 완료된 경우
str_fileCleansed = str('KIER_RAW_' + str_domain + '_Cleansed.csv')
df_raw = pd.read_csv(str_dir_raw + str_fileCleansed, index_col = 0)
print(df_raw.shape, " /// ", df_raw.columns)
df_raw

(33807365, 7)  ///  Index(['METER_DATE', 'HOUSE_ID', 'HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE',
       'HOUSE_ID_HO_INT', 'HOUSE_ID_HO', 'HOT_HEAT_ACCU'],
      dtype='object')


,METER_DATE,HOUSE_ID,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID_HO,HOT_HEAT_ACCU
0,2022-07-17 23:20:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
1,2022-07-17 23:30:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
2,2022-07-17 23:40:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
3,2022-07-17 23:50:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
4,2022-07-18 00:00:06,561-1-1,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=,5.1931000000
...,...,...,...,...,...,...,...
33807360,2024-06-05 14:20:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807361,2024-06-05 14:30:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807362,2024-06-05 14:40:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000
33807363,2024-06-05 14:50:00,563-24-2,563,24,2,ODgwNDU1RUU0RUQ2OTQxQTBGMEI3MTE3NTkxODYxQ0U=,10.3086000000


In [14]:
## 각 세대별 적산값을 파일로 저장
list_h_id = df_kier_hList['HOUSE_ID'].drop_duplicates()

## ▶ Sub-Task 01 : 최적의 Period 계산을 위한, 세대별 Time Period 추출
dt_start, dt_end = 0, 0 ## 최종 시점 및 종점
df_kier_h_Combined = pd.DataFrame() ## 각 호별 Resampled Data 생성 및 취합을 위한 DataFrame
## Sub-Task 01 ◀

for h_id in list_h_id:
    print(h_id)
    df_h_tmp = df_raw[(df_raw['HOUSE_ID'] == h_id)]

    str_col_accu_h = str_col_accu + "_" + h_id

    df_h_tmp['HOUSE_ID_DONG'] = df_h_tmp['HOUSE_ID_DONG'].astype('int').astype('string')
    df_h_tmp['HOUSE_ID_HO_PRE'] = df_h_tmp['HOUSE_ID_HO_PRE'].astype('int').astype('string')
    df_h_tmp['HOUSE_ID_HO'] = df_h_tmp['HOUSE_ID_HO'].astype('string')
    df_h_tmp['HOUSE_ID'] = df_h_tmp['HOUSE_ID'].astype('string')

    df_h_tmp['METER_DATE'] = pd.to_datetime(df_h_tmp['METER_DATE'])
    df_h_tmp = com_date.create_col_ymdhm(df_h_tmp, 'METER_DATE')

    ## METER_DATE의 Minute이 10분 단위가 아닌 경우, 10분 단위로 변경
    for i in range(0, len(df_h_tmp)):
        df_h_tmp['MINUTE'].iloc[i] = (math.floor(df_h_tmp['MINUTE'].iloc[i]/10))*10
    
    int_len_start = len(df_h_tmp)

    ## 변경된 시간값을 기반으로 "METER_DATE" Column을 다시 생성
    ## "METER_DATE"가 중복된 Row를 제거
    ## 1) YMDHM 중, 중복이 있으면 그 중복을 기반으로 Merge 시 유효하지 않은 값들이 발생, Data Row 수가 몇백만 건으로 불어난다.
    ## 2) 적산 Nan값을 제거 : 세대 정보의 Nan값을 복원한 후 dropna()
    df_h_tmp = com_date.create_col_datetime(df_h_tmp, 'METER_DATE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE').drop_duplicates(subset = 'METER_DATE').dropna()

    ## ▶ Sub-Task 01. 각 세대별 Time Period 연산
    if dt_start == 0: 
        dt_start = df_h_tmp['METER_DATE'].min()
        print(dt_start)
    else : 
        if dt_start < df_h_tmp['METER_DATE'].min(): 
            dt_start = df_h_tmp['METER_DATE'].min()
            print("True Start")
    
    if dt_end == 0: 
        dt_end = df_h_tmp['METER_DATE'].max()
        print(dt_end)
    else : 
        if dt_start > df_h_tmp['METER_DATE'].max(): 
            dt_start = df_h_tmp['METER_DATE'].max()
            print("True End")
    ## Sub-Task 01 종료 ◀

    df_h_tmp = df_h_tmp[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu]]
    df_h_tmp = df_h_tmp.rename(columns = {str_col_accu : str_col_accu_h})

    int_len_end = len(df_h_tmp)
    if int_len_start != int_len_end: print(int_len_start, ' / ', int_len_end)

    str_df_h = str('KIER_' + str_domain + '_' + h_id + '_ACCU_01_Raw.csv')
    df_h_tmp.to_csv(str_dirName_h + str_df_h)

print(dt_start, " ~ ", dt_end)
print(df_h_tmp.shape, " /// ", df_h_tmp.columns)
print(df_h_tmp)

561-1-1
2022-07-17 23:20:00
2024-06-05 15:20:00
97285  /  96636
561-1-2
97297  /  96639
561-1-3
97276  /  96616
561-1-4
97044  /  96377
561-2-1
97235  /  96581
561-2-2
97076  /  96423
561-2-3
97278  /  96621
561-2-4
97282  /  96641
561-3-1
97238  /  96597
561-3-2
97291  /  96662
561-3-3
97293  /  96660
561-3-4
97291  /  96615
561-4-1
97252  /  96583
561-4-2
97278  /  96646
561-4-3
97287  /  96650
561-4-4
97236  /  96593
561-5-1
97265  /  96620
561-5-2
97217  /  96573
561-5-3
97253  /  96597
561-5-4
97250  /  96601
561-6-1
97282  /  96593
561-6-2
97275  /  96623
561-6-3
97270  /  96609
561-6-4
97267  /  96619
561-7-1
97270  /  96607
561-7-2
97245  /  96593
561-7-3
97249  /  96596
561-7-4
97285  /  96619
561-8-1
97211  /  96558
561-8-2
97291  /  96630
561-8-3
97287  /  96617
561-8-4
97303  /  96668
561-9-1
97291  /  96656
561-9-2
97237  /  96569
561-9-3
97301  /  96651
561-9-4
97257  /  96590
561-10-1
97259  /  96556
561-10-2
97247  /  96570
561-10-3
97308  /  96635
561-10-4
97294  /  96

### 적산 사용량

#### 모든 호실의 사용량을 변수화하여 한 데이터셋에 Combine

In [15]:
## 여기서부터 실행하려 할 경우, Period를 지정해줘야함
## 정상적인 Time Period 외의 데이터는 삭제 (2022-07-17 ~ 2024-06-05)
df_period_tmp = pd.read_csv(str_dir_raw + "KIER_TIME_PERIOD.csv", index_col = 0)
df_period_tmp['METER_DATE'] = pd.to_datetime(df_period_tmp['METER_DATE'])

## 선행 계산된 Period(최종 시점 및 종점)에 대한 Date Dataframe 생성
df_kier_h_Combined = com_date.create_df_dt(pd.DataFrame(), 'METER_DATE'
                                           , df_period_tmp['METER_DATE'].min(), df_period_tmp['METER_DATE'].max(), '10min')[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']]

## METER_DATE의 Minute이 10분 단위가 아닌 경우, 10분 단위로 변경
for i in range(0, len(df_kier_h_Combined)) : df_kier_h_Combined['MINUTE'].iloc[i] = (math.floor(df_kier_h_Combined['MINUTE'].iloc[i]/10))*10
df_kier_h_Combined

,YEAR,MONTH,DAY,HOUR,MINUTE
0,2022,7,17,23,0
1,2022,7,17,23,10
2,2022,7,17,23,20
3,2022,7,17,23,30
4,2022,7,17,23,40
...,...,...,...,...,...
99166,2024,6,5,14,40
99167,2024,6,5,14,50
99168,2024,6,5,15,0
99169,2024,6,5,15,10


In [16]:
for h_id in list_h_id:
    print(h_id)
    str_df_h = str('KIER_' + str_domain + '_' + h_id + '_ACCU_01_Raw.csv')
    df_h_tmp = pd.read_csv(str_dirName_h + str_df_h, index_col = 0).reset_index()
    # df_h_tmp['METER_DATE'] = pd.to_datetime(df_h_tmp['METER_DATE'])

    str_col_accu_h = str_col_accu + "_" + h_id
    df_h_tmp = df_h_tmp[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', str_col_accu_h]]

    df_kier_h_Combined = pd.merge(df_kier_h_Combined, df_h_tmp
                                    , how = 'left', on = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'])
    print(df_kier_h_Combined.shape)
df_kier_h_Combined

561-1-1
(99171, 6)
561-1-2
(99171, 7)
561-1-3
(99171, 8)
561-1-4
(99171, 9)
561-2-1
(99171, 10)
561-2-2
(99171, 11)
561-2-3
(99171, 12)
561-2-4
(99171, 13)
561-3-1
(99171, 14)
561-3-2
(99171, 15)
561-3-3
(99171, 16)
561-3-4
(99171, 17)
561-4-1
(99171, 18)
561-4-2
(99171, 19)
561-4-3
(99171, 20)
561-4-4
(99171, 21)
561-5-1
(99171, 22)
561-5-2
(99171, 23)
561-5-3
(99171, 24)
561-5-4
(99171, 25)
561-6-1
(99171, 26)
561-6-2
(99171, 27)
561-6-3
(99171, 28)
561-6-4
(99171, 29)
561-7-1
(99171, 30)
561-7-2
(99171, 31)
561-7-3
(99171, 32)
561-7-4
(99171, 33)
561-8-1
(99171, 34)
561-8-2
(99171, 35)
561-8-3
(99171, 36)
561-8-4
(99171, 37)
561-9-1
(99171, 38)
561-9-2
(99171, 39)
561-9-3
(99171, 40)
561-9-4
(99171, 41)
561-10-1
(99171, 42)
561-10-2
(99171, 43)
561-10-3
(99171, 44)
561-10-4
(99171, 45)
561-11-1
(99171, 46)
561-11-2
(99171, 47)
561-11-3
(99171, 48)
561-11-4
(99171, 49)
561-12-1
(99171, 50)
561-12-2
(99171, 51)
561-12-3
(99171, 52)
561-12-4
(99171, 53)
561-13-1
(99171, 54)
561-13-2
(9

,YEAR,MONTH,DAY,HOUR,MINUTE,HOT_HEAT_ACCU_561-1-1,HOT_HEAT_ACCU_561-1-2,HOT_HEAT_ACCU_561-1-3,HOT_HEAT_ACCU_561-1-4,HOT_HEAT_ACCU_561-2-1,...,HOT_HEAT_ACCU_563-22-3,HOT_HEAT_ACCU_563-22-4,HOT_HEAT_ACCU_563-22-5,HOT_HEAT_ACCU_563-22-6,HOT_HEAT_ACCU_563-23-1,HOT_HEAT_ACCU_563-23-2,HOT_HEAT_ACCU_563-23-3,HOT_HEAT_ACCU_563-23-4,HOT_HEAT_ACCU_563-24-1,HOT_HEAT_ACCU_563-24-2
0,2022,7,17,23,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,7,17,23,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,7,17,23,20,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,...,11.0474000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2486000000,6.6785000000,4.8390000000,5.7506000000
3,2022,7,17,23,30,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,...,11.0476000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2486000000,6.6785000000,4.8390000000,5.7506000000
4,2022,7,17,23,40,5.1931000000,11.7151000000,6.8699000000,7.6715000000,8.0506000000,...,11.0477000000,7.6167000000,4.7248000000,2.9381000000,3.2043000000,2.6466000000,4.2487000000,6.6785000000,4.8390000000,5.7506000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99166,2024,6,5,14,40,11.4979000000,22.7068000000,16.7914000000,18.4898000000,16.6083000000,...,22.4070000000,13.6466000000,10.4224000000,5.5183000000,7.6796000000,7.2011000000,9.2295000000,14.8264000000,11.8592000000,10.3086000000
99167,2024,6,5,14,50,11.4979000000,22.7068000000,16.7918000000,18.4902000000,16.6083000000,...,22.4070000000,13.6466000000,10.4224000000,5.5183000000,7.6796000000,7.2011000000,9.2295000000,14.8265000000,11.8592000000,10.3086000000
99168,2024,6,5,15,0,11.4979000000,22.7068000000,16.7918000000,18.4902000000,16.6083000000,...,NaN,NaN,10.4224000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99169,2024,6,5,15,10,11.4979000000,22.7068000000,16.7918000000,18.4902000000,16.6083000000,...,22.4070000000,13.6466000000,10.4224000000,5.5183000000,7.6796000000,7.2011000000,9.2295000000,14.8265000000,11.8592000000,10.3086000000


In [17]:
list_col = df_kier_h_Combined.columns[6:]
# df_dt_period = df_kier_h_Combined[['METER_DATE']]

df_kier_Calc = df_kier_h_Combined[list_col]
df_kier_extract = df_kier_h_Combined

df_kier_extract['MEAN_OF_ACCU'], df_kier_extract['SUM_OF_ACCU'] = df_kier_Calc.mean(axis = 1), df_kier_Calc.sum(axis = 1)

str_file = 'KIER_' + str_domain + '_ACCU_10MIN.csv'
df_kier_extract.to_csv(str_dirName_h + str_file)